In [1]:
from secretnote.compat.secretflow.device.driver import SFConfigSimulationFullyManaged

secretflow_config = SFConfigSimulationFullyManaged(parties=["alice", "bob"])

In [2]:
import secretflow

secretflow.shutdown()
secretflow.init(**secretflow_config.dict())

2023-10-09 13:50:53,966	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-09 13:51:00,910	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [3]:
alice = secretflow.PYU("alice")
bob = secretflow.PYU("bob")

In [4]:
from secretnote.compat.spu import (
    SPUConfig,
    SPUClusterDef,
    SPUNode,
    SPUProtocolKind,
    SPUFieldType,
    SPURuntimeConfig,
)

spu_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(party="alice", address="localhost:32767"),
            SPUNode(party="bob", address="localhost:32768"),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    ),
)

In [5]:
spu = secretflow.SPU(**spu_config.dict())

In [6]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from secretnote.instrumentation import Profiler
from secretnote.instrumentation.rules import DEFAULT_RULES


resource = Resource(attributes={SERVICE_NAME: "simulation"})
provider = TracerProvider(resource=resource)
provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint="localhost:4317", insecure=True)),
)
trace.set_tracer_provider(provider)

In [7]:
import jax.numpy as jnp
import re


def dot(x, y):
    return jnp.dot(x, y)


with Profiler(DEFAULT_RULES):
    x = secretflow.to(alice, jnp.asarray([1, 2, 3]))
    y = secretflow.to(bob, jnp.asarray([1, 2, 3]))
    z = x.to(spu)
    w = y.to(spu)
    r = spu(dot)(z, w)
    s = r.to(alice)
    secretflow.reveal(s)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=68932) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=68932) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=68932) INFO:jax._src.xla_bridge:Unable to initialize back